<a href="https://colab.research.google.com/github/kiplangatkorir/MetaLearning/blob/main/geological_regions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/conedata.csv')
data.head()

Mounted at /content/drive


,ID,test_type,basin_valley,Depth (m),qc (MPa),fs (kPa),u2 (kPa),Vs (m/s),qt (MPa),Rf (%),...,Fr (%),Bq (-),U2 (-),SBT (-),SBTn (-),Mod. SBTn (-),n,Ic (-),Ic SBT (-),Ib (-)
0,0,CPT,Zell basin,0.02,1.15,1.0,0,0,1.77,0.06,...,0.06,0,0,0,0,0,-0.12,2.56,2.23,25.61
1,0,CPT,Zell basin,0.04,2.85,1.0,0,0,2.99,0.03,...,0.03,0,0,0,0,0,0.33,1.23,2.02,256.60
2,0,CPT,Zell basin,0.06,4.80,1.0,0,0,4.76,0.06,...,0.06,0,0,0,0,0,0.28,1.16,1.80,264.66
3,0,CPT,Zell basin,0.08,6.40,6.0,0,0,6.12,0.07,...,0.07,0,0,0,0,0,0.27,1.12,1.69,275.17
4,0,CPT,Zell basin,0.10,6.90,6.0,0,0,6.91,0.10,...,0.10,0,0,0,0,0,0.27,1.11,1.65,267.92


In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


In [36]:

# Features
X = data[['Depth (m)', 'qc (MPa)', 'fs (kPa)', 'u2 (kPa)', 'Vs (m/s)']]

# Target variables
y = data[['SBT (-)', 'Ic SBT (-)']]

In [38]:
# Split data into train and test sets by geological region
groups = data['basin_valley']
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


In [46]:
# Function to create a deep neural network model
def create_dnn_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [49]:
# Define base models
base_models = [
    RandomForestRegressor(),
    XGBRegressor(),
]


In [50]:
# Train each base model on the training data
trained_models = []
for model in base_models:
    if isinstance(model, Sequential):
        model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)
    else:
        model.fit(X_train, y_train)
    trained_models.append(model)

In [51]:
# Make predictions on the test data using each base model
predictions = []
for model in trained_models:
    predictions.append(model.predict(X_test))

In [52]:

# Create a meta-dataset by combining the predictions from the base models
meta_X = np.column_stack(predictions)
meta_y = y_test.values


In [53]:
# Train a meta-model on the meta-dataset
meta_model = LinearRegression()
meta_model.fit(meta_X, meta_y)

LinearRegression()

In [61]:
# Evaluate the meta-model's performance
meta_predictions = meta_model.predict(meta_X)
mse = mean_squared_error(meta_y, meta_predictions)
r2 = r2_score(meta_y, meta_predictions)
print(f"Meta-model MSE: {mse:.3f}")
print(f"Meta-model R^2 score: {r2:.3f}")

Meta-model MSE: 0.076
Meta-model R^2 score: 0.806
